In [7]:
# The first step is to import the libraries necessary to perform the mathematical operations and the keras data set 
import numpy as np 
import pandas as pd 
from tqdm import tqdm 
from keras.datasets import mnist 

# this loads the data from the mnist data set 
(xtrain, ytrain), (xtest,xtest) = mnist.load_data()

#this selects a subset of the mnist data (150 images)
xtrain=xtrain[:300]
y=ytrain[:300]

X=xtrain.T
X=X/255

y.resize((300,1))
y=y.T

#check the values 
pd.Series(y[0]).value_counts()

#this output shows 10 categories for the data, for this CNN we want two categories so we need to convert it to binary
for i in range(y.shape[1]):
    if y[0][i]> 4:
        y[0][i]=1
    else:
        y[0][i]=0
#Check values     
pd.Series(ytrain[0]).value_counts()

#The data has now been classified as either a 1 or a 0

#The next step is to create filters in this case 4 and we will randomize the filter values 
f=np.random.uniform(size=[5,5,4])
#printing the filters and data set sizes c, helpful to see image size and check filter size is what you expect 
    #for i in range(4):
        #print ('Filter', i) 
        #print( '\n',f[:,:,i],'\n')

    #X.shape, y.shape, f.shape
    
#The next task is to break the images into sections that are the size of the filters (5,5) 
#this prepares the data for convolution 

#this  is list or the sections of each image that are the size of the filter
imsect=[]

#iterates through each image
for i in range(X.shape[2]): 
    #image size-filter size+1 is the number of slices of the image the filter will take 
    #horizontal traveral
    for j in range(X.shape[0]-f.shape[0]+1):
        #vertical traversal 
        for k in range(X.shape[1]-f.shape[1]+1): 
            imsect.append(X[:,:,i][j:j+f.shape[0],k:k+f.shape[1]])
    #Now we have make a list of these images, it will be easier to work with in the future if we can reshape this list into 
    #an array 
imsect=np.array(imsect)
imsect.resize(X.shape[2],int(imsect.shape[0]/X.shape[2]),imsect.shape[1], imsect.shape[2])

#After this it is necessary to create the weight matrix 

#first create parameters for the data input variables
rowsize=X.shape[0]-f.shape[0]+1
colsize=X.shape[1]-f.shape[1]+1
filternum=f.shape[2]

#the total number of input cells or neurons 
inputlayercells=(rowsize)*(colsize)*(filternum)
output=1

wo=np.random.uniform(size=(inputlayercells,output)) 

#Next we need to define the activation function and its derivative 
#the function used here is the sigmoid function, often used for binary classification problems

def act(x):
    return 1/(1+np.exp(-x))
def actder(x):
    return x*(1-x)
#Next we apply to filters to perform the layers of convolution 

conv_out=[]
#the number of images 
for i in range(len(imsect)):
    #the number of filters 
    for j in range(f.shape[2]):
    #to apply the filter we multiply element-wise 
    #this first loop iterates through one dimension, with vectorized code performing the rest of the iterating
        for k in range(imsect.shape[1]): 
            conv_out.append((f[:,:,j]*imsect[i][k]).sum())
conv_out=np.array(conv_out)
conv_out.resize(len(imsect),f.shape[2],imsect.shape[1])
conv_outact=act(conv_out)

#at this point the shape is (number of images, number of filters, number of sections per image)
#the next step of the CNN is a fully connected layer, which is essentially a classic neural network 
##this requires a 1D array of input, which means the output needs to be reshaped 
conv_outact=conv_outact.reshape(conv_out.shape[0],conv_out.shape[1]*conv_out.shape[2])

#this transposes the matrix to facilitate the linear transformation 
conv_outact=conv_outact.T 

#linear transformation 
dotpr=np.dot(wo.T,conv_outact)
out_in=(dotpr-np.average(dotpr))/np.sd(dotpr)

#apply acitvation function 
out=act(out_in)

##up to this point, the process has been forward propogation
##the input was convoluted/filtered, and outputted 
##backward propogation is used to make models like this more acurrate
##To begin with backward propogation we have to assess the current models error 

#Error 
error=np.square(y-out)/2

#Error with respect to output (slope)
error_slope=-(y-out)

#Error with respect to transformation (out_in)
error_transf=out*(1-out)

#Error with weight 
error_wgt=conv_outact 



    